<font size="5">**OpenAI GPT Basic Example**</font>

This is a basic example of how to use the 
OpenAI GPT API in Python. This Jupyter
notebook was designed to run in Google Colab,
but can easily work on other platforms as well.

In this example we construct a prompt that utilizes
a number of variables that describe a forest stand in the 
formulation of a response.The prompt informs the model of 
an average value for basal area that it my comment on 
when drafting the stand description.

William Zipse, NJ Forest Service 2023

In [1]:
#Set API Key
#You will need an OpenAI API key to run this script!
#Use your own key where quoted in the API_KEY constant below.
#This will establish which key to charge for tokens!
#BE CAREFUL!!!
API_KEY = ""

<font size="3">**Some notes about calling OpenAI from Python**</font>

The OpenAI documentation gives examples of loading the API key either for all projects or for a single project. OpenAI recommends using the methods for setting up the key for all projects, setting up an environment variable locally, and calling it in code without including the key. This is not always practical, of course.

The other recommended method involves setting up the key for a single project using a .env file that contains the key and calling that file from one's code. The documentation then recommends adding the .env file to .gitignore, if using git version control. This way the key is never included in the code or uploaded to a repository.

The code used in this example is meant as a demonstration, so you can put your API key straight into the API_KEY constant listed at the beginning of this file with minimal setup and see how the interface works. NEVER save your key in this example or any code you are working on! If the file is copied or uploaded, you risk publicly exposing a paid API usage key.

The code below is used to install the openai Python library
in the Google Colab environment

In [ ]:
#install OpenAI
#Uncomment below if using Google Colab
#comment out if using a local install
#!pip install --upgrade openai

In [2]:
#Taken from OpenAI documentation
from openai import OpenAI

In [ ]:
#Establish paths and connections

In [3]:
#Set API Key from API_KEY constant set at the beginning of this notebook
client = OpenAI(api_key=API_KEY)

<font size="3">**Variables for the Prompt**</font>

The variables below contain the values that will be used in
the forest stand description by the GPT model. In this example
the values are set in the definitions below, however these 
variables could easily be changed in other implementations.

In [4]:
#define forest stand variables
qmd = 6.0
tpa = 311
ba = 95
ft = 'Pitch Pine/Oak'
area = 150

<font size="3">**Making the Prompt**</font>

The string variable called "p1" below contains the prompt to send to the openai API. 
Note that the variables defined previously are referred to in this prompt. Also 
note that prompts are preceded by "Prompt:" and responses are preceded by "Response:" 
In this case, the response after "Response:"is left blank for the model to complete.
You may give the model a series of defined prompts and responses prior to leaving a
response blank. The delimiters of "Prompt:" and "Response:" are defined in the call 
to client.chat.completions.create() in the block after the definition of the variable "p1".

In [5]:
p1 = '''Prompt:
Generate a description of a forest stand where 
QMD is quadratic mean diameter in inches, TPA is trees per acre, BA is average tree basal 
area in square feet per acre, FT is forest type naming dominant tree species in the overstory, 
and area is the stand area in
acres. '''+'In this stand QMD = '+str(qmd)+', '+'TPA = '+str(tpa)+', BA = '+str(ba)+', FT = '+str(ft)+'area = '+str(area)+'''
The statewide average BA is 111.0 square feet per acre. \n Response:'''

<font size="3">**Calling the OpenAI API**</font>

Below the variable "response" is defined by calling the 
openai.Completion.create() function.  The function returns 
nested Python dictionaries.  Model parameters can be set here. 
See OpenAI documentation for details. Note the "prompt" variable 
is set to "p" above; max_tokens can be adjusted to adjust the 
length of prompts and responses (larger values may charge 
more tokens on the OpenAI account associated with the API key);
temperature can be set between 0 - 2. This is were the "stop" 
list is set as well. Here it is set to "Prompt:" for the prompt 
and "Response:" for the response."

In [7]:
#set model parameters here
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a forestry assistant, that explains sceintific concepts as a professional forester."},
    {"role": "user", "content": p1}
  ]
)


<font size="3">**Locating the Text Response**</font>

Below the variable "answer" is set to the location of the 
text response from the "response" variable above.

In [9]:
answer = completion.choices[0].message

<font size="3">**Let's see the text response!**</font>

Here we print the "answer" variable generated by the model.

Now you can try changing the prompt and variables above and
see what kinds of results you can get.

In [10]:
print(answer)

ChatCompletionMessage(content="In this forest stand, we observe a diverse mix of tree species dominated by Pitch Pine and Oak. The quadratic mean diameter (QMD) of the trees is measured at 6.0 inches, indicating a range of tree sizes within the stand. The stand has a tree density of 311 trees per acre (TPA), indicating a relatively high density of trees. The average tree basal area (BA) is calculated at 95 square feet per acre, slightly below the statewide average of 111.0 square feet per acre.\n\nWith an area of 150 acres, this forest stand is a significant tract of land supporting a mix of Pitch Pine and Oak as the dominant tree species. The QMD, TPA, and BA values provide insights into the stand's structure and composition, indicating a varied and dense stand with a mix of tree sizes.\n\nOverall, this forest stand with its specific QMD, TPA, and BA values, along with the dominance of Pitch Pine and Oak, contributes to the biodiversity and ecological richness of the area. Understandi

**Let's try using our prompt in a slightly different way.**

In the example above we defined one text prompt in the variable, p. This prompt
has only one defined prompt after the stop sequence defined with the string
of "Prompt:". We then follow the prompt with the stop sequence of "Response:" and allow the model to generate the response.

This format works for relatively small prompts because prompts and responses have limited numbers of tokens (aka characters) that can be processed. It is possible to construct a prompt containing a series of prompts and responses
before requesting a response from the model, allowing more tokens to be
processed in a prompt sequence.  the prompt stored in the variable "p2" is an
example of defining a prompt sequence this way.

Also note that we setup a function here called askGPT that calls the same type of completion syntax we used earlier, but
makes it a little simpler if we have to call multiple completions. the function takes a prompt (p), followed by the 
description of an assisstant (a).

In [11]:
#setup a function if you want to pass many prompts
#to a model setup the same way for each prompt
#the function takes prompt, p and assistant
#description a
def askGPT(p,a):
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role":"system","content":a},
          {"role":"user","content":p}
        ]
      )
  return response

**A prompt with prompts and responses**

Notice that although the prompt defined in the variable "p2" below prompts the 
same information as the prompt as that at "p1," this prompt spreads the information across multiple prompts and responses. This allows for the 
construction of longer prompts with more background information.

In [12]:
p2 = '''Prompt: What does QMD stand for?
Response: QMD is quadratic mean diamer in inches.\n
Prompt: What does TPA stand for?\n
Response: TPA stands for trees per acre.\n
Prompt: What does BA stand for?\n
Response: BA stands for average forest stand basal area in square feet per acre.\n
Prompt: What is the statewide average BA for reference?\n
Response: The statewide average BA is 111.0 square feet per acre.\n
Prompt: What is area?\n
Response: area is the size of the forest stand in acres.\n
Prompt: What is FT?\n
Response: FT is the forest type, which names the dominant tree species in the overstory.\n
Prompt: Generate a description of a forest stand where
'''+'in this stand QMD = '+str(qmd)+', '+'TPA = '+str(tpa)+', BA = '+str(ba)+', FT = '+str(ft)+'area = '+str(area)+'''
Mention how the stand BA compares to the statewide BA\n Response:'''

In [13]:
assistant = "You are a forestry assistant, that explains sceintific concepts as a professional forester."

In [14]:
#call our function to run the model
response = askGPT(p2, assistant)

In [15]:
answer2 = response.choices[0].message

Once again we have a response that was left blank in p2, generated by the model.

In [16]:
print(answer2)

ChatCompletionMessage(content='In this stand, the quadratic mean diameter (QMD) is 6.0 inches, the trees per acre (TPA) is 311, and the average forest stand basal area (BA) is 95 square feet per acre. The dominant tree species in the overstory is Pitch Pine/Oak, and the forest stand covers an area of 150 acres.\n\nComparing the stand BA of 95 square feet per acre to the statewide average BA of 111.0 square feet per acre, we can see that this particular stand has a slightly lower basal area than the statewide average. This may indicate a slightly less dense or mature forest stand in comparison to the average across the state.', role='assistant', function_call=None, tool_calls=None)
